# Week 5 - Milestone 1: Smart Tracking System Blockchain Ledger (Submission)

1. Open your Python script and load the CSV file from your Homework 1:

In [10]:
import pandas as pd


# Load IoT sensor data from CSV (Generated in Homework 1)
df = pd.read_csv("data/smart_tracking_data.csv")

3. Connect Python to the smart contract.
 - Ensure Ganache is running in the background.
 - Establish connection with Web3.py:


In [11]:
from web3 import Web3


# Connect to local blockchain
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# Verify connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")

✅ Connected to Ganache successfully!


- Load the smart contract:

In [12]:
# Replace with actual contract address from Remix
contract_address = "0x069af8b9480dcb0001eb618a72d1ff57c3e350de"


# Paste the ABI from Remix
import json

abi_json = '''
[
	{
		"inputs": [],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"anonymous": false,
		"inputs": [
			{
				"indexed": false,
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "deviceId",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "dataType",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "dataValue",
				"type": "string"
			}
		],
		"name": "DataStored",
		"type": "event"
	},
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "_deviceId",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_dataType",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_dataValue",
				"type": "string"
			}
		],
		"name": "storeData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "dataRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "deviceId",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "dataType",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "dataValue",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getRecord",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "getTotalRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "MAX_ENTRIES",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "owner",
		"outputs": [
			{
				"internalType": "address",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]'''


# Now load it safely as a Python object
abi = json.loads(abi_json)


# Convert to Python-compatible dict
abi = json.loads(abi_json)


# Ensure contract address is in checksum format
contract_address_checksum = web3.to_checksum_address(contract_address)

# Load the smart contract
contract = web3.eth.contract(address=contract_address_checksum, abi=abi)


# Set the default sender address (first account from Ganache)
web3.eth.default_account = web3.eth.accounts[0]


print(f"✅ Connected to Smart Contract at {contract_address}")

✅ Connected to Smart Contract at 0x069af8b9480dcb0001eb618a72d1ff57c3e350de


4. Send the IoT data to the blockchain. Each row from the CSV file will be stored as a transaction on the blockchain.
 - Define a function to store IoT data:

In [13]:
import time


def send_iot_data(device_id, data_type, data_value):
    """Sends IoT data to the deployed smart contract"""
    txn = contract.functions.storeData(device_id, data_type, data_value).transact({
        'from': web3.eth.default_account,
        'gas': 3000000
    })
   
    receipt = web3.eth.wait_for_transaction_receipt(txn)
    print(f"✅ Data Stored: {data_type} - {data_value}, Txn Hash: {receipt.transactionHash.hex()}")

 - Loop through the CSV file and send each row to the blockchain:

In [17]:
for _, row in df.iterrows():
    # Example: send temperature as data_type and its value as data_value
    send_iot_data(str(row["device_id"]), str(row["temperature"]), str(row["weight"]))
    time.sleep(1)  # Delay to prevent flooding transactions

✅ Data Stored: 7.79 - 1.97, Txn Hash: fe64fb163b5e38ccd8ac3b9fddab8a07cee735f187fd9ceaabc609f89b19031a
✅ Data Stored: 9.55 - 14.29, Txn Hash: 905bb062554e75a27719504c17925578cbd83b8c2f5f47133c49fd981dad7fb1
✅ Data Stored: 4.82 - 15.78, Txn Hash: 0e26445152bd15270091ed6b29ff455e4f7afe3dc00aefc16211fc44cb4fe330
✅ Data Stored: 4.81 - 9.33, Txn Hash: c7be5f138ccd828320cfc435583a598a78729f6fab355becb01f1807bca30a3d
✅ Data Stored: 5.76 - 16.98, Txn Hash: 36db9ce51dab72a4346e9234a48e3e80cd8a419870516feb937d681096790e50
✅ Data Stored: 6.44 - 11.31, Txn Hash: b591295cec4d7a6fa64ff2faeb5c86613bbe1ae6a7a5a093f261d310d51e5e75
✅ Data Stored: 4.91 - 3.44, Txn Hash: ab36ee179fd90951bdb456581960082440a48bb6a3462176d4fb3b41fa92d51e
✅ Data Stored: 7.22 - 7.25, Txn Hash: 6e1ee3a05443fc356e2eace03f8f5640f48c87f50df1a1e5873a9bfff41d708c
✅ Data Stored: 3.32 - 5.68, Txn Hash: 939ac83a239ad99658e45788359ccc8667e6bb37af7c2faf31718f38ee8be18e
✅ Data Stored: 3.48 - 23.21, Txn Hash: 369e2c27358921195e5a2410f7312a

5. Now that the data is on the blockchain, retrieve it to verify storage.
 - Get total stored records:

In [18]:
total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

Total IoT records stored: 195


 - Retrieve and print a specific record

In [19]:
record = contract.functions.getRecord(0).call()
print("First Stored Record:", record)

First Stored Record: [1750311908, 'DVC776', 'temperature', '1.97']
